# Read H5 files to csv
Read h5 files created in Data_extration.ipynb

In [10]:
import os
import pandas as pd
from pathlib import Path
from harp_resources import process, utils
import h5py
#from analysis_functions import *

In [11]:
mouse_info = {'B2M1': {'sex': 'M', 'area': 'V2M'},
              'B2M4': {'sex': 'M', 'area': 'V2M'},
              'B2M5': {'sex': 'M', 'area': 'V2M'},
              'B2M6': {'sex': 'M', 'area': 'V2M'},
              'B3M1': {'sex': 'M', 'area': 'V2M'},
              'B3M2': {'sex': 'M', 'area': 'V2M'},
              'B3M3': {'sex': 'F', 'area': 'V1'},
              'B3M4': {'sex': 'M', 'area': 'V2M'},
              'B3M5': {'sex': 'M', 'area': 'V2M'},
              'B3M6': {'sex': 'F', 'area': 'V2M'},
              'B3M7': {'sex': 'F', 'area': 'V2M'},
              'B3M8': {'sex': 'F', 'area': 'V2M'},
              'B0M0': {'sex': 'F', 'area': 'V2M'},
             }

session_info = {'220824': 'day1',
                '230824': 'day2',
                '190824': 'day1',
                '200824': 'day2',
                '120824': 'day1',
                '130824': 'day2',
                '070824': 'day1',
                '080824': 'day2',
               }

#'SleapVideoData2'] = ['Ellipse.Diameter', 'Ellipse.Center.X', 'Ellipse.Center.Y']

## Defining paths for grab or G8

In [12]:
rootdir = '/Volumes/RanczLab/20240730_Mismatch_Experiment/G8_MMclosed-and-Regular_130824'#Enter root path
rootdir = '/Users/nora/Desktop/Cohort0_GCaMP_example/2024-08-08T10-05-26_B3M3'
h5_paths = []
eventpaths = []
for dirpath, subdirs, files in os.walk(rootdir):
    for x in files:
        if '.h5' in x:
            eventpaths.append(dirpath)
            h5_paths.append(dirpath+'/'+x)

In [13]:
h5_paths

['/Users/nora/Desktop/Cohort0_GCaMP_example/2024-08-08T10-05-26_B3M3/Old_h5file/resampled_streams_2024-08-08T10-05-26_B3M3.h5',
 '/Users/nora/Desktop/Cohort0_GCaMP_example/2024-08-08T10-05-26_B3M3/deleteme/resampled_streams_2024-08-08T10-05-26_B3M3.h5']

In [14]:
# Expression unit testing
'''h5_paths = ['/Volumes/RanczLab/20240730_Mismatch_Experiment/Fake_data_080824/Test_streams_B0M0/resampled_streams_Test_streams_B0M0.h5',
            '/Volumes/RanczLab/20240730_Mismatch_Experiment/Fake_data_070824/Test_streams_B0M0/resampled_streams_Test_streams_B0M0.h5']
eventpaths = ['/Volumes/RanczLab/20240730_Mismatch_Experiment/Fake_data_080824/Test_streams_B0M0',
             '/Volumes/RanczLab/20240730_Mismatch_Experiment/Fake_data_070824/Test_streams_B0M0']'''

"h5_paths = ['/Volumes/RanczLab/20240730_Mismatch_Experiment/Fake_data_080824/Test_streams_B0M0/resampled_streams_Test_streams_B0M0.h5',\n            '/Volumes/RanczLab/20240730_Mismatch_Experiment/Fake_data_070824/Test_streams_B0M0/resampled_streams_Test_streams_B0M0.h5']\neventpaths = ['/Volumes/RanczLab/20240730_Mismatch_Experiment/Fake_data_080824/Test_streams_B0M0',\n             '/Volumes/RanczLab/20240730_Mismatch_Experiment/Fake_data_070824/Test_streams_B0M0']"

### Loading data streams

In [15]:

import numpy as np
def load_h5_streams_to_dict(data_paths):
    '''
    Takes list of H5 file paths and, loads streams into dictionary, and save to dictionary named by mouse ID
    '''
    reconstructed_dict = {}  # Dictionary to save streams
    
    for input_file in data_paths:
        
        name = input_file.split('/')[-1][-7:-3]  # Extract mouse ID from file name
        
        if not os.path.exists(input_file):
            print(f'ERROR: {input_file} does not exist.')
            return None
        
        with h5py.File(input_file, 'r') as h5file:
            print(f'reconstructing streams for mouse {name}, from session folder: {input_file.split("/")[-3]}')
            
            common_index = h5file['HARP_timestamps'][:]
            reconstructed_streams = {}
            
            for source_name in h5file.keys():
                if source_name == 'HARP_timestamps':
                    continue
                
                reconstructed_streams[source_name] = {}
                source_group = h5file[source_name]
                
                for stream_name in source_group.keys():
                    stream_data = source_group[stream_name][:]
                    length_difference = len(common_index) - len(stream_data)
                    
                    # Pad or truncate to match common_index length
                    if len(stream_data) < len(common_index):
                        padding = np.full(len(common_index) - len(stream_data), np.nan)
                        stream_data = np.concatenate([stream_data, padding])
                        print(f"{source_name} - {stream_name}: Length difference: {length_difference}")
                        print(f"missing data, advicable to ensure correct alignment \n ")
                    elif len(stream_data) > len(common_index):
                        stream_data = stream_data[:len(common_index)]
                    
                    reconstructed_streams[source_name][stream_name] = pd.Series(data=stream_data, index=common_index)
        if name not in reconstructed_dict.keys():   
            reconstructed_dict[name] = reconstructed_streams
            print(f'  --> {name} streams reconstructed and added to dictionary \n')
        else: 
            reconstructed_dict[f'{name}_2'] = reconstructed_streams
            print(f'  --> {name} streams_2 reconstructed and added to dictionary \n')
    
    return reconstructed_dict




In [16]:
stream_dict_dict = load_h5_streams_to_dict(h5_paths)

reconstructing streams for mouse B3M3, from session folder: 2024-08-08T10-05-26_B3M3
  --> B3M3 streams reconstructed and added to dictionary 

reconstructing streams for mouse B3M3, from session folder: 2024-08-08T10-05-26_B3M3
  --> B3M3 streams_2 reconstructed and added to dictionary 



In [17]:
stream_dict_dict.keys()

dict_keys(['B3M3', 'B3M3_2'])

In [18]:
stream_dict_dict['B3M3']#['Photometry']

{'H1': {'OpticalTrackingRead0X(46)': 768958.9499    0.000000
  768958.9500    0.000824
  768958.9501    0.001648
  768958.9502    0.002472
  768958.9503    0.003296
                   ...   
  770875.9411    0.000399
  770875.9412    0.000299
  770875.9413    0.000200
  770875.9414    0.000100
  770875.9415    0.000000
  Length: 19169917, dtype: float64,
  'OpticalTrackingRead0Y(46)': 768958.9499    0.000000
  768958.9500    0.000340
  768958.9501    0.000680
  768958.9502    0.001021
  768958.9503    0.001361
                   ...   
  770875.9411    0.000005
  770875.9412    0.000004
  770875.9413    0.000002
  770875.9414    0.000001
  770875.9415    0.000000
  Length: 19169917, dtype: float64},
 'H2': {},
 'ONIX': {'Photodiode': 768958.9499    True
  768958.9500    True
  768958.9501    True
  768958.9502    True
  768958.9503    True
                 ... 
  770875.9411    True
  770875.9412    True
  770875.9413    True
  770875.9414    True
  770875.9415    True
  Length: 191699

In [19]:
#Make a cut_info dict for the mouse with missing data
#cut_info = {'B2M5': 521, 'B3M2':174}

In [20]:
def make_dataframes(stream_dict_dict, cut_info = {}):
    data_dict = {}
    for mouse, streamdict in stream_dict_dict.items():
        
        print(f'\n--Making dataframe for {mouse}--')
        #Getting fluorescence traces
        try: 
            fluorescence = streamdict['Photometry']['470_dfF'] #Using '470_dfF' only
        except KeyError:
            fluorescence = streamdict['Photometry']['CH1-470']
        print('flourescence 470 extracted')
    
        #Getting mouse movement data and converting to cm / second
        movementX = process.running_unit_conversion(streamdict['H1']['OpticalTrackingRead0X(46)'])*100
        movementY = process.running_unit_conversion(streamdict['H1']['OpticalTrackingRead0Y(46)'])*100
        print('movement on x and Y axis extracted')
    
        #Getting eye movements and pupil diameter 
        #'SleapVideoData2' = ['Ellipse.Diameter', 'Ellipse.Center.X', 'Ellipse.Center.Y']
        if 'SleapVideoData2' in streamdict:
            eye_center_x = streamdict['SleapVideoData2']['Ellipse.Center.X']
            eye_center_y = streamdict['SleapVideoData2']['Ellipse.Center.Y']
            eye_diameter = streamdict['SleapVideoData2']['Ellipse.Diameter']
            print('eye movement data extracted')
        else: 
            print('There was no eye movement data available for ', mouse)
    
        #Getting visual stimuli event times
        event = streamdict['ONIX']['Photodiode']
        print('photdiode halt info extracted')
        
        time = movementX.index - movementX.index[0]
        print('time in seconds from 0 extracted form X direction movement')
        
        dict = {'470_dfF': fluorescence, 'movementX': movementX, 'movementY': movementY, 'event': event,
            'Seconds': time}
        #dict = {'470_dfF': fluorescence, 'movementX': movementX, 'movementY': movementY, 'event': event,
         #   'TimeStamp': time, 'eye_x': eye_center_x, 'eye_y': eye_center_y, 'pupil_diameter': eye_diameter}
        
        df = pd.DataFrame(dict)
        print('dataframe created with columns: ', df.columns)
        #if mouse in cut_info:
            #df = df.iloc[:-cut_info[mouse]]
        
        df['event'] = df['event'].astype(bool) #In case column is not bool
        #Reversing, so that a halt appearst when 'event'==True
        df['event'] = ~df['event']
        print('Event column as bool, True values corresponding to halts')
        
        df.reset_index(inplace=False)
        
        data_dict[mouse]= df
    return data_dict

In [21]:
def make_dataframes(stream_dict_dict, fluorescense_traces = ['470_dfF'], cut_info={}):
    data_dict = {}
    for mouse, streamdict in stream_dict_dict.items():
        
        print(f'\n--Making dataframe for {mouse}--')
        # Getting fluorescence traces
        fluorescence_dict = {}
        for trace in fluorescense_traces:
            try: 
                fluorescence_dict[trace] = streamdict['Photometry'][trace] 
            except KeyError:
                print(f'Trace {trace} not available')
                pass
    
        # Getting mouse movement data and converting to cm / second
        movementX = process.running_unit_conversion(streamdict['H1']['OpticalTrackingRead0X(46)']) * 100
        movementY = process.running_unit_conversion(streamdict['H1']['OpticalTrackingRead0Y(46)']) * 100
        print('movement on x and Y axis extracted')
    
        # Getting eye movements and pupil diameter
        # 'SleapVideoData2' = ['Ellipse.Diameter', 'Ellipse.Center.X', 'Ellipse.Center.Y']
        eye_data_available = 'SleapVideoData2' in streamdict
        if eye_data_available:
            eye_center_x = streamdict['SleapVideoData2']['Ellipse.Center.X']
            eye_center_y = streamdict['SleapVideoData2']['Ellipse.Center.Y']
            eye_diameter = streamdict['SleapVideoData2']['Ellipse.Diameter']
            print('eye movement data extracted')
        else: 
            print('There was no eye movement data available for ', mouse)
    
        # Getting visual stimuli event times
        event = streamdict['ONIX']['Photodiode']
        print('photodiode halt info extracted')
        
        time = movementX.index - movementX.index[0]
        print('time in seconds from 0 extracted from X direction movement')
        
        # Creating the dictionary for the DataFrame
        data_dict_for_df = {
            'movementX': movementX, 
            'movementY': movementY, 
            'event': event,
            'Seconds': time
        }
        data_dict_for_df.update(fluorescence_dict)
        
        # Adding eye data to the dictionary if available
        if eye_data_available:
            data_dict_for_df['eye_x'] = eye_center_x
            data_dict_for_df['eye_y'] = eye_center_y
            data_dict_for_df['pupil_diameter'] = eye_diameter
        
        df = pd.DataFrame(data_dict_for_df)
        print('dataframe created with columns: ', df.columns)
        
        # if mouse in cut_info:
        #     df = df.iloc[:-cut_info[mouse]]
        
        df['event'] = df['event'].astype(bool)  # In case column is not bool
        # Reversing, so that a halt appears when 'event'==True
        df['event'] = ~df['event']
        print('Event column as bool, True values corresponding to halts')
        
        df.reset_index(inplace=False)
        
        data_dict[mouse] = df
    return data_dict


In [22]:
data_dict = make_dataframes(stream_dict_dict, fluorescense_traces = ['470_dfF', 'z_470'])
names = [name for name in data_dict]


--Making dataframe for B3M3--
Trace z_470 not available
movement on x and Y axis extracted
eye movement data extracted
photodiode halt info extracted
time in seconds from 0 extracted from X direction movement
dataframe created with columns:  Index(['movementX', 'movementY', 'event', 'Seconds', '470_dfF', 'eye_x',
       'eye_y', 'pupil_diameter'],
      dtype='object')
Event column as bool, True values corresponding to halts

--Making dataframe for B3M3_2--
Trace 470_dfF not available
Trace z_470 not available
movement on x and Y axis extracted
eye movement data extracted
photodiode halt info extracted
time in seconds from 0 extracted from X direction movement
dataframe created with columns:  Index(['movementX', 'movementY', 'event', 'Seconds', 'eye_x', 'eye_y',
       'pupil_diameter'],
      dtype='object')
Event column as bool, True values corresponding to halts


In [23]:
data_dict[names[0]]

,movementX,movementY,event,Seconds,470_dfF,eye_x,eye_y,pupil_diameter
768958.9499,0.000000,0.000000,False,0.0000,0.000000,-27.143936,-16.524869,13.642569
768958.9500,0.001744,0.000720,False,0.0001,-0.000016,-27.143856,-16.525130,13.642585
768958.9501,0.003488,0.001440,False,0.0002,-0.000031,-27.143775,-16.525392,13.642602
768958.9502,0.005232,0.002161,False,0.0003,-0.000047,-27.143695,-16.525653,13.642618
768958.9503,0.006976,0.002881,False,0.0004,-0.000063,-27.143615,-16.525915,13.642634
...,...,...,...,...,...,...,...,...
770875.9411,0.000845,0.000010,False,1916.9912,-0.049615,-0.000156,-0.000399,0.000488
770875.9412,0.000634,0.000007,False,1916.9913,-0.047973,-0.000117,-0.000300,0.000366
770875.9413,0.000422,0.000005,False,1916.9914,-0.046330,-0.000078,-0.000200,0.000244
770875.9414,0.000211,0.000002,False,1916.9915,-0.044687,-0.000039,-0.000100,0.000122


In [24]:
for mouse, df in data_dict.items():
    percent_true = (len(df.loc[df['event']==True])*100)/len(df)
    print(f'for {mouse} the True values makes up {percent_true:.2f} % of the total df lenght' )
    if percent_true > 50:
        print('This is more than 50 %, which may be too much, consider inversing True/False or check experiment protocol for mouse')

for B3M3 the True values makes up 1.35 % of the total df lenght
for B3M3_2 the True values makes up 1.35 % of the total df lenght


### Loading Experiment events and session info

In [43]:
eventpath = '/Users/nora/Desktop/Cohort0_GCaMP_example/2024-08-08T10-05-26_B3M3/'
eventpath.split('/')[-2].split('_')[-1]

'B3M3'

In [40]:
event_dict = {}
for eventpath in eventpaths:
    eventpath = '/Users/nora/Desktop/Cohort0_GCaMP_example/2024-08-08T10-05-26_B3M3/'
    ExpEvents = process.read_ExperimentEvents(Path(eventpath))
    ExpEvents.set_index('Seconds', inplace = True)
    ExpEvents.index = ExpEvents.index.round(4)
    name = eventpath.split('/')[-1][-4:]
    ExpEvents['experiment'] = eventpath.split('/')[-2].split('_')[-1]
    for key, item in session_info.items():
        if key in eventpath.split('/')[-2]:
            ExpEvents['session']=item
    if name not in event_dict.keys():  
        event_dict[name] = ExpEvents
    else:
        event_dict[f'{name}_2'] = ExpEvents


In [44]:
event_dict['B3M3'] = event_dict['']

In [48]:
event_dict['B3M3']['experiment']='closedopenMM'

### Adding events (and non-events) and session info to data
1) add_experiment_events() takes the event data, and inserts the values for every timepoint into an event column in the main data
* Sometimes, there are multiple event strings during the same timepoint. Then, if a crucial event takes place during this, this one will be priotized
* The crucial events can be defined from strings they contain in the first line of the add_experiment_events function.
* In no crucial event are at that timepoint, all the events will be assinged to the timepoint in the main df as one string, seperated by a comma (use .split(',') later if it becomes necessary to seperate them during analysis)

2) The No_halt events are used to make a column where the no-halt events are used to make a bool similar to the 'event' (halt) column
   * These can later be used as control as they appear when there could have been a halt but there was none
   * The number of no-halt events is controlled to ensure that all of them were actually used.

In [49]:
def add_experiment_events(data_dict, events_dict, mouse_info):
    # Iterate over each mouse key in the dictionaries
    for mouse_key in data_dict:
        # Retrieve the main and event DataFrames
        main_df = data_dict[mouse_key]
        event_df = events_dict[mouse_key]

        # Ensure both indices are sorted
        main_df = main_df.sort_index()
        event_df = event_df.sort_index()

        # Perform a merge_asof on the index to add 'Value' as 'ExperimentEvents' with backward matching
        merged_df = pd.merge_asof(
            main_df,
            event_df[['Value']],  # Only select the 'Value' column from event_df
            left_index=True,
            right_index=True,
            direction='backward',
            tolerance=0  # Adjust tolerance for matching on the index
        )

        # Rename the 'Value' column to 'ExperimentEvents'
        if 'ExperimentEvents' in merged_df.columns:
            merged_df['ExperimentEvents'] = merged_df.pop('Value')  # Replace existing column with the new 'Value' column
            print(f'Pre-existing ExperimentEvents column was replaced with new for {mouse_key}')
        else:
            merged_df = merged_df.rename(columns={'Value': 'ExperimentEvents'})  # Add new column
            print(f'Added new ExperimentEvents for {mouse_key}')

        # Add metadata from event_df
        #merged_df['Experiment'] = event_df['experiment'].unique()[0]
        #merged_df['Session'] = event_df['session'].unique()[0]

        # Add mouse ID, sex, and brain area
        mouse_info_name = mouse_key[:4]
        merged_df['mouseID'] = mouse_info_name
        merged_df['sex'] = mouse_info[mouse_info_name]['sex']
        merged_df['area'] = mouse_info[mouse_info_name]['area']

        # Update the dictionary with the merged DataFrame
        data_dict[mouse_key] = merged_df

    return data_dict

In [51]:
data_dict = add_experiment_events(data_dict, event_dict,mouse_info)

Added new ExperimentEvents for B3M3


KeyError: 'B3M3_2'

In [52]:
data_dict[names[0]].ExperimentEvents.unique() #Check random mouse to see what events are present

array([nan, 'Sync signal started', 'LinearMismatch block started',
       'Homing platform', 'Wait for run threshold...',
       'Halt delay: 0.141714071082749s', 'Apply halt: 1s',
       'Halt delay: 0.428817241279789s', 'Halt delay: 0.208678958196509s',
       'Halt delay: 0.554045762752204s', 'Halt delay: 0.280655684871904s',
       'Halt delay: 0.581960399533604s', 'Halt delay: 0.140750847217045s',
       'Halt delay: 0.563808195415795s', 'Halt delay: 0.472889200399113s',
       'Halt delay: 0.176535663603123s', 'Halt delay: 0.461901821969963s',
       'Halt delay: 0.413050603639824s', 'Halt delay: 0.262688852363587s',
       'Halt delay: 0.306834821592474s', 'Halt delay: 0.346818170299203s',
       'Halt delay: 0.3154486012717s', 'Halt delay: 0.463464505115275s',
       'Halt delay: 0.457007012635938s', 'Halt delay: 0.221602529483662s',
       'LinearPlaybackMismatch block started',
       'Halt delay: 0.211700286209444s', 'Halt delay: 0.446463341194421s',
       'Halt delay: 0.20

In [53]:
data_dict = process.add_no_halt_column(data_dict, event_dict)

No_halt events added to B3M3
  Correct number of no-halt events for B3M3



KeyError: 'B3M3_2'

#### Add block columns
For each mouse and corresponding df, update the df to include columns for each block of the experiment for easy slicing later in analysis.
The add_block_columns() function will also test if each of the created block columns contains at least one True value and that there are no temporal overlaps 

In [54]:
for name, df in data_dict.items():
    print('\n updating data for ', name,'...')
    blocks_added_df = process.add_block_columns(df, event_dict[name])
    blocks_added_df.replace({})
    data_dict[name] = blocks_added_df

process.check_block_overlap(data_dict)


 updating data for  B3M3 ...
LinearMismatch block started
LinearPlaybackMismatch block started

 updating data for  B3M3_2 ...


KeyError: 'B3M3_2'

In [60]:
process.check_block_overlap(data_dict)

For B3M3: No overlapping True values, and each _block column has at least one True value
For B3M3_2: No overlapping True values, and each _block column has at least one True value


### The downsample_data function can be used to make the datset smaller. 
1) Make an empty dict to fill with downsampled versions of the dfs.
2) Loop though the mice and dfs, and use the function for each
3) Set the name of the time column to use and decide on the frequency of the output df datapoints
        * Ensure that all the columns that you want to keep has a corresponding dict key in aggregation_functions in the downsample_data() funciton. 
4) Assign the resulting df to the corresponding mouse
5) test_event_numbers as a way to test if all events (no-halt events as they are frequent, can be changed) survived the downsampling. 

NB: Can be slow with large datasets, check your email

In [55]:
def downsample_data(df, time_col='Seconds', interval=0.001):
    '''
    Uses pandas resample and aggregate functions to downsample the data to the desired interval. 
    * Note: Aggregation functions must be applied for each variable that is to be included.
    https://pandas.pydata.org/docs/reference/api/pandas.core.resample.Resampler.aggregate.html
    * Note: because the donwsampling keeps the first non-NaN value in each interval, some values could be lost.
    '''
    # Convert the Seconds column to a TimedeltaIndex
    df = df.set_index(pd.to_timedelta(df[time_col], unit='s'))

    #define aggregation functions for all possible columns
    aggregation_functions = {
        '470_dfF': 'mean', # takes the mean signal of the datapoints going into each new downsampled datapoint
        '560_dfF': 'mean',
        'movementX': 'mean',
        'movementY': 'mean',
        'event': 'any', # events column is a bool, and if there is any True values in the interval, the downsampled datapoint will be True
        'ExperimentEvents': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None, #first non-NaN value in the interval 
        'Experiment': 'first', # All values should be the same, so it can always just take the first string value
        'Session': 'first',
        'mouseID': 'first',
        'sex': 'first',
        'area': 'first',
        'No_halt': 'any', 
        'LinearMismatch_block': 'any', 
        'LinearPlaybackMismatch_block': 'any',
        'LinearRegular_block': 'any',
        'LinearClosedloopMismatch_block':'any',
        'LinearRegularMismatch_block':'any',
        'LinearNormal_block':'any',
    }

    # Filter aggregation_functions to only include columns present in df
    aggregation_functions = {key: func for key, func in aggregation_functions.items() if key in df.columns}

    print('downsampling...')
    # Resample with the specified interval and apply the filtered aggregations
    downsampled_df = df.resample(f'{interval}s').agg(aggregation_functions)

    # Reset the index to make the Seconds column normal again
    downsampled_df = downsampled_df.reset_index()
    downsampled_df[time_col] = downsampled_df[time_col].dt.total_seconds()  # Convert Timedelta back to seconds

    # Forward fill for categorical columns if needed, only if they exist in downsampled_df
    categorical_cols = ['Experiment', 'Session', 'mouseID', 'sex', 'area']
    for col in categorical_cols:
        if col in downsampled_df.columns:
            downsampled_df[col] = downsampled_df[col].ffill()

    # Remove consecutive duplicate values in the 'ExperimentEvents' column, if it exists
    if 'ExperimentEvents' in downsampled_df.columns:
        downsampled_df['ExperimentEvents'] = downsampled_df['ExperimentEvents'].where(
            downsampled_df['ExperimentEvents'] != downsampled_df['ExperimentEvents'].shift()
        )

    return downsampled_df

In [56]:
def test_event_numbers(downsampled_data, original_data, mouse):
    '''
    Counts number of True values in the No_halt columns in the original and the downsampled data
    This will indicate whether information was lost in the downsampling.
    If the original events somehow has been upsampled previously (for example if the tolerance was set too high in add_experiment_events()), 
    repeatings of the same event can also lead to fewer True events in the downsampled df.
    '''
    nohalt_down = len(downsampled_data.loc[downsampled_data['No_halt']==True])
    nohalt_original = len(original_data.loc[original_data['No_halt']==True])
    if nohalt_down != nohalt_original:
        print(f'mouse{mouse}')
        print(f'There are actually {nohalt_original} no-halts, but the downsampled data only contains {nohalt_down}')
        print('Should re-run the downsampling. Try changing interval lenght. Othewise, consider not downsampling\n')
    if nohalt_down == nohalt_original:
        print(f'mouse{mouse}')
        print(f'There are {nohalt_original} no-halts, and downsampled data contains {nohalt_down}\n')


In [58]:
downsampled_dict = {}
for mouse, df in data_dict.items():
    downsampled_df = process.downsample_data(df, time_col='Seconds', interval=0.001)
    downsampled_dict[mouse] = downsampled_df
    process.test_event_numbers(downsampled_df, df, mouse)


downsampling...
mouseB3M3
There are 60 no-halts, and downsampled data contains 60

downsampling...


KeyError: 'No_halt'

In [59]:

downsampled_dict[names[0]].loc[downsampled_dict[names[0]].No_halt == True]

,Seconds,470_dfF,movementX,movementY,event,ExperimentEvents,mouseID,sex,area,No_halt,LinearMismatch_block,LinearPlaybackMismatch_block
38365,38.365,-2.495511,12.191956,-2.193510,False,Wait for run threshold...,B3M3,F,V1,True,True,False
56698,56.698,-1.795482,10.968392,8.442896,False,Wait for run threshold...,B3M3,F,V1,True,True,False
78390,78.390,-2.662937,13.306828,3.868227,False,Wait for run threshold...,B3M3,F,V1,True,True,False
151100,151.100,-0.082036,13.163146,5.272895,False,Wait for run threshold...,B3M3,F,V1,True,True,False
198910,198.910,2.481018,14.781736,3.811240,False,Wait for run threshold...,B3M3,F,V1,True,True,False
239441,239.441,5.854799,16.444188,2.567010,False,Wait for run threshold...,B3M3,F,V1,True,True,False
257941,257.941,-1.106507,1.150983,-0.286222,False,Wait for run threshold...,B3M3,F,V1,True,True,False
272190,272.190,-2.176610,20.770837,9.388583,False,Wait for run threshold...,B3M3,F,V1,True,True,False
285573,285.573,1.738420,23.460751,1.202644,False,Wait for run threshold...,B3M3,F,V1,True,True,False
300223,300.223,-0.140659,15.531779,5.218731,False,Wait for run threshold...,B3M3,F,V1,True,True,False


### Concat and reindex

In [61]:
All_data = pd.concat([Data for Data in downsampled_dict.values()])


# Set a file name and save

!!! Make sure to change file save names before running the below cell

In [62]:
All_data

,Seconds,470_dfF,movementX,movementY,event,ExperimentEvents,mouseID,sex,area,No_halt,LinearMismatch_block,LinearPlaybackMismatch_block
0,0.000,-0.000071,0.007848,0.003241,False,None,B3M3,F,V1,False,False,False
1,0.001,-0.000228,0.025288,0.010443,False,None,B3M3,F,V1,False,False,False
2,0.002,-0.000385,0.042729,0.017645,False,None,B3M3,F,V1,False,False,False
3,0.003,-0.000542,0.060169,0.024846,False,None,B3M3,F,V1,False,False,False
4,0.004,-0.000699,0.077609,0.032048,False,None,B3M3,F,V1,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1916976,1916.976,NaN,0.008562,0.000101,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1916977,1916.977,NaN,0.006448,0.000076,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1916978,1916.978,NaN,0.004334,0.000051,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1916979,1916.979,NaN,0.002220,0.000026,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:

if 'day1' in All_data.Session.values:
    All_data.to_csv('Mismatch_analysis/G8_MMclosed_regular_session1.csv', index=False) #Change name
if 'day2' in All_data.Session.values:
    All_data.to_csv('Mismatch_analysis/G8_MMclosed_regular_session2.csv', index=False) #Change name
    


In [ ]:
All_data

In [66]:
downsampled_dict['B3M3'].to_csv('Mismatch_analysis/B3M3_G8_MMclosed_session1.csv', index=False) #Change name